In [14]:
import pandas as pd
import plotly.graph_objects as go

In [17]:
log_file = './cpu.log'

with open(log_file) as fd:
    lines = fd.readlines()

load_headers = ['user', 'nice', 'sys', 'idle', 'iow', 'irq', 'sirq']
headers = load_headers + ['time_value']
cpu_load_dict = {}
    
cnt = 0
while cnt < len(lines) - 6:
    time = pd.to_datetime(lines[cnt])
    cnt+=2
    
    for i in range(4):
        cpu_num = f'cpu{i}'
        if cpu_num not in cpu_load_dict:
            cpu_load_dict[cpu_num] = dict(zip(headers, [list() for _ in headers]))
        
        splitted_line = lines[cnt].split()
        cpu_load_dict[cpu_num]['user'].append(int(splitted_line[1]))
        cpu_load_dict[cpu_num]['nice'].append(int(splitted_line[2]))
        cpu_load_dict[cpu_num]['sys'].append(int(splitted_line[3]))
        cpu_load_dict[cpu_num]['idle'].append(int(splitted_line[4]))
        cpu_load_dict[cpu_num]['iow'].append(int(splitted_line[5]))
        cpu_load_dict[cpu_num]['irq'].append(int(splitted_line[6]))
        cpu_load_dict[cpu_num]['sirq'].append(int(splitted_line[7]))
        cpu_load_dict[cpu_num]['time_value'].append(time)
        cnt+=1

df_dict = {}
for key, val in cpu_load_dict.items():
    per_cpu_df = pd.DataFrame(val)
    
    per_cpu_df[load_headers] = per_cpu_df[load_headers].diff()
    per_cpu_df.dropna(inplace=True)
    
    per_cpu_df['sum'] = per_cpu_df[load_headers].sum(axis=1)
    
    per_cpu_df[load_headers] = per_cpu_df[load_headers].div(per_cpu_df['sum'], axis=0).mul(100)
    
    per_cpu_df['load'] = per_cpu_df['idle'].mul(-1).add(100)
    
    per_cpu_df.drop('sum', axis=1, inplace=True)
    
    df_dict[key] = per_cpu_df


fig = go.Figure()
for cpu, val in cpu_load_dict.items():
    df = df_dict[cpu]
    fig.add_trace(go.Scatter(x=df['time_value'], y=df['load'], name=cpu))

fig.update_layout(
#     width=800,
#     height=800,
    title='CPU load',
    yaxis_title='%',
    xaxis_title='Time'
)

fig.show()